Based on: https://github.com/rhasspy/piper/blob/master/notebooks/piper_inference_(ONNX).ipynb

In [1]:
%cd /content
print("Installing...")
!git clone -q https://github.com/rmcpantoja/piper
%cd /content/piper/src/python
!pip install -q piper-phonemize==1.1.0 librosa>=0.9.2 numpy>=1.19.0 onnxruntime>=1.11.0
!pip install -q onnxruntime-gpu pydub
import os
if not os.path.exists("/content/piper/src/python/lng"):
  !cp -r "/content/piper/notebooks/lng" /content/piper/src/python/lng
import sys
sys.path.append('/content/piper/notebooks')
print("Success!")

/content
Installing...
/content/piper/src/python
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.5/291.5 MB 4.7 MB/s eta 0:00:00
Success!


In [2]:
!wget https://huggingface.co/rhasspy/piper-voices/resolve/main/hu/hu_HU/berta/medium/hu_HU-berta-medium.onnx?download=true -O hu_HU-berta-medium.onnx
!wget https://huggingface.co/rhasspy/piper-voices/resolve/main/hu/hu_HU/berta/medium/hu_HU-berta-medium.onnx.json?download=true -O hu_HU-berta-medium.onnx.json
!wget https://huggingface.co/rhasspy/piper-voices/resolve/main/en/en_US/lessac/high/en_US-lessac-high.onnx?download=true -O en_US-lessac-high.onnx
!wget https://huggingface.co/rhasspy/piper-voices/resolve/main/en/en_US/lessac/high/en_US-lessac-high.onnx.json?download=true -O en_US-lessac-high.onnx.json


--2024-11-07 12:33:40--  https://huggingface.co/rhasspy/piper-voices/resolve/main/hu/hu_HU/berta/medium/hu_HU-berta-medium.onnx?download=true
Resolving huggingface.co (huggingface.co)... 18.164.174.55, 18.164.174.17, 18.164.174.118, ...
Connecting to huggingface.co (huggingface.co)|18.164.174.55|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.hf.co/repos/ed/06/ed062eb100d7bd80d78f61252cd190fca48cbda97eb1753fb827ff3339a6b11c/4eed05f767573b77fd2c07e6bccaa9b3c77089a55b9239c3099ecd3d17a59be3?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27hu_HU-berta-medium.onnx%3B+filename%3D%22hu_HU-berta-medium.onnx%22%3B&Expires=1731242020&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczMTI0MjAyMH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy9lZC8wNi9lZDA2MmViMTAwZDdiZDgwZDc4ZjYxMjUyY2QxOTBmY2E0OGNiZGE5N2ViMTc1M2ZiODI3ZmYzMzM5YTZiMTFjLzRlZWQwNWY3Njc1NzNiNzdmZDJjMDdlNmJjY2FhOWIzYzc3MDg5Y

In [3]:
 use_gpu=False

In [5]:
%cd /content/piper/src/python
# original: infer_onnx.py
import json
import logging
import math
import sys
from pathlib import Path
from enum import Enum
from typing import Iterable, List, Optional, Union
import numpy as np
import onnxruntime
from piper_train.vits.utils import audio_float_to_int16
import glob
import ipywidgets as widgets
from IPython.display import display, Audio, Markdown, clear_output
from piper_phonemize import phonemize_codepoints, phonemize_espeak, tashkeel_run
import time
from pydub import AudioSegment
import scipy
style = {'description_width': 'initial'}

_LOGGER = logging.getLogger("piper_train.infer_onnx")

def detect_onnx_models(path):
    onnx_models = glob.glob(path + '/*.onnx')
    if len(onnx_models) > 1:
        return onnx_models
    elif len(onnx_models) == 1:
        return onnx_models[0]
    else:
        return None

def wav2mp3(path_to_file):
    final_audio = AudioSegment.from_wav(file=path_to_file)
    path_to_file = path_to_file.replace(".wav",".mp3")
    final_audio.export(path_to_file, format="mp3")
    return path_to_file

def main():
    """Main entry point"""
    models_path = "/content/piper/src/python"
    logging.basicConfig(level=logging.DEBUG)
    providers = [
        "CPUExecutionProvider"
        if use_gpu is False
        else ("CUDAExecutionProvider", {"cudnn_conv_algo_search": "DEFAULT"})
    ]
    sess_options = onnxruntime.SessionOptions()
    model = None
    onnx_models = detect_onnx_models(models_path)
    speaker_selection = widgets.Dropdown(
        options=[],
        description=f'{"Select speaker"}:',
        layout={'visibility': 'hidden'}
    )
    if onnx_models is None:
        raise Exception( "No downloaded voice packages!")
    elif isinstance(onnx_models, str):
        onnx_model = onnx_models
        model, config = load_onnx(onnx_model, sess_options, providers)
        if config["num_speakers"] > 1:
            speaker_selection.options = config["speaker_id_map"].values()
            speaker_selection.layout.visibility = 'visible'
            preview_sid = 0
        else:
            speaker_selection.layout.visibility = 'hidden'
            preview_sid = None

    else:
        voice_model_names = []
        for current in onnx_models:
            voice_struct = current.split("/")[5]
            voice_model_names.append(voice_struct)
        selection = widgets.Dropdown(
            options=voice_model_names,
            description=f'{"Select voice package"}:',
        )
        load_btn = widgets.Button(
            description="Load it!"
        )
        config = None
        def load_model(button):
            nonlocal config
            global onnx_model
            nonlocal model
            nonlocal models_path
            selected_voice = selection.value
            onnx_model = f"{models_path}/{selected_voice}"
            model, config = load_onnx(onnx_model, sess_options, providers)
            if config["num_speakers"] > 1:
                speaker_selection.options = config["speaker_id_map"].values()
                speaker_selection.layout.visibility = 'visible'
            else:
                speaker_selection.layout.visibility = 'hidden'
            display(Markdown(f'Model loaded: {selected_voice}'))

        load_btn.on_click(load_model)
        display(selection, load_btn)
    display(speaker_selection)
    speed_slider = widgets.FloatSlider(
        value=1,
        min=0.25,
        max=4,
        step=0.1,
        description="Rate scale",style = style,
        orientation='horizontal',
    )
    noise_scale_slider = widgets.FloatSlider(
        value=0.667,
        min=0.25,
        max=4,
        step=0.1,
        description= "Phoneme noise scale",style = style,
        orientation='horizontal',
    )
    noise_scale_w_slider = widgets.FloatSlider(
        value=1,
        min=0.9,
        max=1.1,
        step=0.1,
        description="Phoneme stressing scale",style = style,
        orientation='horizontal',
    )
    sentence_silence  = widgets.Dropdown(
        options=[
            0.0,
            0.5,
            1.0,
            2.0,
            3.0,
            5.0
        ],
        value=0.0,
        description="Silence between sentences",style = style,
    )
    play = widgets.Checkbox(
        value=True,
        description="Auto-play",
        disabled=False
    )
    mp3 = widgets.Checkbox(
        value=False,
        description="Save to mp3",
        disabled=False
    )
    text_input = widgets.Text(
        value='',
        placeholder=f'{"Enter your text here"}:',
        description="Text to synthesize", style = style,
        layout=widgets.Layout(width='70%')
    )
    synthesize_button = widgets.Button(
        description= "Synthesize",
        button_style='success', # 'success', 'info', 'warning', 'danger' or ''
        tooltip="Click here to synthesize the text.",
        icon='check'
    )
    close_button = widgets.Button(
        description="Exit",
        tooltip="Closes this GUI.",
        icon='check'
    )

    def on_synthesize_button_clicked(b):
        if model is None:
            raise Exception("You have not loaded any model from the list!")
        text = text_input.value
        if config["num_speakers"] > 1:
            sid = speaker_selection.value
        else:
            sid = None
        rate = speed_slider.value
        noise_scale = noise_scale_slider.value
        noise_scale_w = noise_scale_w_slider.value
        silence = sentence_silence.value
        auto_play = play.value
        savemp3 = mp3.value
        inferencing(
            model,
            config,
            sid,
            text,
            rate,
            noise_scale,
            noise_scale_w,
            silence,
            auto_play,
            savemp3
        )

    def on_close_button_clicked(b):
        clear_output()


    synthesize_button.on_click(on_synthesize_button_clicked)
    close_button.on_click(on_close_button_clicked)
    display(text_input)
    display(speed_slider)
    display(noise_scale_slider)
    display(noise_scale_w_slider)
    display(sentence_silence)
    display(play)
    display(mp3)
    display(synthesize_button)
    display(close_button)
    display(Markdown(f'## {"Audio history"}'))

def load_onnx(model, sess_options, providers = ["CPUExecutionProvider"]):
    _LOGGER.debug("Loading model from %s", model)
    config = load_config(model)
    model = onnxruntime.InferenceSession(
        str(model),
        sess_options=sess_options,
        providers= providers
    )
    _LOGGER.info("Loaded model from %s", model)
    return model, config

def load_config(model):
    with open(f"{model}.json", "r") as file:
        config = json.load(file)
    return config
PAD = "_"  # padding (0)
BOS = "^"  # beginning of sentence
EOS = "$"  # end of sentence

class PhonemeType(str, Enum):
    ESPEAK = "espeak"
    TEXT = "text"

def phonemize(config, text: str) -> List[List[str]]:
    """Text to phonemes grouped by sentence."""
    if config["phoneme_type"] == PhonemeType.ESPEAK:
        if config["espeak"]["voice"] == "ar":
            # Arabic diacritization
            # https://github.com/mush42/libtashkeel/
            text = tashkeel_run(text)
        return phonemize_espeak(text, config["espeak"]["voice"])
    if config["phoneme_type"] == PhonemeType.TEXT:
        return phonemize_codepoints(text)
    raise ValueError(f'Unexpected phoneme type: {config["phoneme_type"]}')

def phonemes_to_ids(config, phonemes: List[str]) -> List[int]:
    """Phonemes to ids."""
    id_map = config["phoneme_id_map"]
    ids: List[int] = list(id_map[BOS])
    for phoneme in phonemes:
        if phoneme not in id_map:
            print("Missing phoneme from id map: %s", phoneme)
            continue
        ids.extend(id_map[phoneme])
        ids.extend(id_map[PAD])
    ids.extend(id_map[EOS])
    return ids

def inferencing(model, config, sid, line, length_scale = 1, noise_scale = 0.667, noise_scale_w = 0.8, sentence_silence = 0.0, auto_play=True, save_mp3 = False):
    audios = []
    if config["phoneme_type"] == "PhonemeType.ESPEAK":
        config["phoneme_type"] = "espeak"
    text = phonemize(config, line)
    num_silence_samples = int(sentence_silence * config["audio"]["sample_rate"])
    num_speakers = config["num_speakers"]
    silence = np.zeros(num_silence_samples, dtype=np.int16)
    for phonemes in text:
        phoneme_ids = phonemes_to_ids(config, phonemes)
        if num_speakers == 1:
            speaker_id = None # for now
        else:
            speaker_id = sid
        text = np.expand_dims(np.array(phoneme_ids, dtype=np.int64), 0)
        text_lengths = np.array([text.shape[1]], dtype=np.int64)
        scales = np.array(
            [noise_scale, length_scale, noise_scale_w],
            dtype=np.float32,
        )
        sid = None
        if speaker_id is not None:
            sid = np.array([speaker_id], dtype=np.int64).flatten()
        audio = model.run(
            None,
            {
                "input": text,
                "input_lengths": text_lengths,
                "scales": scales,
                "sid": sid,
            },
        )[0].squeeze((0, 1))
        audio = audio_float_to_int16(audio.squeeze())
        audio = np.concatenate((audio, silence))
        audios.append(audio)
    merged_audio = np.concatenate(audios)
    sample_rate = config["audio"]["sample_rate"]
    if not len(line) > 2000:
        display(Markdown(f"{line}"))
    else:
        display(Markdown("Too long text"))
    if save_mp3:
        timestr = time.strftime("%Y%m%d-%H%M%S")
        new_audio_name = f"{timestr}.wav"
        scipy.io.wavfile.write(new_audio_name, sample_rate, merged_audio)
        converted = wav2mp3(new_audio_name)
        display(Audio(converted, rate=sample_rate, autoplay=auto_play))
    else:
        display(Audio(merged_audio, rate=sample_rate, autoplay=auto_play))


main()

/content/piper/src/python


Dropdown(description='Select voice package:', options=('hu_HU-berta-medium.onnx', 'en_US-lessac-high.onnx'), v…

Button(description='Load it!', style=ButtonStyle())

Dropdown(description='Select speaker:', layout=Layout(visibility='hidden'), options=(), value=None)

Text(value='', description='Text to synthesize', layout=Layout(width='70%'), placeholder='Enter your text here…

...

Button(button_style='success', description='Synthesize', icon='check', style=ButtonStyle(), tooltip='Click her…

Button(description='Exit', icon='check', style=ButtonStyle(), tooltip='Closes this GUI.')

## Audio history

Model loaded: hu_HU-berta-medium.onnx

Ez egy példamondat.